<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import sys
import os
from pathlib import Path
import networkx as nx
from numba import jit
from dask.distributed import Client
pd.options.mode.chained_assignment = None  # default='warn'

client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/124/1,Cluster Workers: 1 Cores: 2 Memory: 13.65 GB




---


**Tentative code for rnc2 temporal network**

@time

Conditions -> Boolean mask -> Contacts -> Edgelist


---



In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/Academia/01_Publications/00_Alex/005_RNC2')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [0]:
wind_data

In [0]:
def wind_scenario(wind_data):
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance

**Main**


---



In [0]:
%%time
n=2
for scenario in range(n):
    # initial setup
    condition = True
    list_of_Activations = []
    time = 0 
    
    # wind conditions
    w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
    
    # ignition
    rng = np.random.uniform(0, 1, size=edgelist.values.shape[0])
    CoTime = pd.DataFrame ((rng < edgelist.IgnProb_bl.values)*1, 
                           columns=['initial_state'])
    print("number of ignitions : {}".format(len(CoTime[CoTime.initial_state == 1])))
    
    # add scenario and time to active edges
    ActiveEdges = edgelist[CoTime['initial_state'] == 1]
    ActiveEdges["scenario"] = scenario
    ActiveEdges["time"] = time
    list_of_Activations.append(ActiveEdges)
    
    while condition:
        print("scenario : {} time : {}".format(scenario, time))
        
        # propagation mask
        maskPreviousEdge = CoTime.iloc[:, -1] == 1 
        newSource = edgelist.target[maskPreviousEdge]
        maskSource = np.in1d(edgelist.source.values, newSource.values)
        # print("shape maskSource ; {}".format(maskSource.shape))
        
        # wind mask
        maskWind = (edgelist.bearing.values < w_bearing_max) & \
            (edgelist.bearing.values > w_bearing_min) & \
                (edgelist.distance.values < w_distance)
        # print("shape maskWind ; {}".format(maskWind.shape))
        
        # burnt mask
        previouslyActivated = CoTime.iloc[:, :-1].sum(axis=1) # all 0 for time == 0, intact
        maskBurnt = np.where(previouslyActivated> 0, 0, 1) # burnt == 0, intact == 1(mask needed)
        # print("shape maskBurnt ; {}".format(maskBurnt.shape))
        
        # store mask in CoTime matrix
        maskMerge = (maskSource) & (maskWind) & (maskBurnt)
        CoTime['time{}'.format(time)] = maskMerge
        # print("shape maskMerge ; {}".format(maskMerge.shape))
        
        # Active edges at this time
        ActiveEdges = edgelist[CoTime['time{}'.format(time)] == 1]
        ActiveEdges["scenario"] = scenario
        ActiveEdges["time"] = time
        list_of_Activations.append(ActiveEdges)
        
        # break while condition if no more fires
        fires = (CoTime['time{}'.format(time)] == 1).sum(axis=0)
        condition = fires != 0
        print("condition {} at time {}".format(condition, time))
        
        # advance time
        time += 1
        
    # if while loop broken, save activation for the scenario
    Activations = pd.concat(list_of_Activations)
    Activations.to_parquet(folder / 'output' / 'scenario{}_Activations.parquet'.format(scenario), engine='pyarrow')